In [1]:
import torch
import PIL
from PIL import Image
import torchvision.transforms as transforms
import cv2
import numpy as np
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from matplotlib import pyplot as plt
import time
import easyocr
import datetime
# import yolov5.utils
# from yolov5.utils.general import non_max_suppression

/home/tcestudent/anaconda3/envs/cnn/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
currentSpeedLimit = 500
cascade_src = '../Speed_Detection_CV/cascade/classifier/cascade.xml'
video_src = "../../integration/testing.mp4"
conf = 0.25
nms_threshold = 0.5

In [3]:

def YoloDetection(model,imgs):
    if(type(imgs)==str):
        imgs = Image.open(imgs)
        imgs = torch.stack([transform(imgs)])
    elif(type(imgs)==np.ndarray):
        res = model(imgs)
        resList = res.pandas().xyxy[0]
        return resList.values[:,:6].tolist()
    elif(type(imgs)==torch.Tensor):
        pass
    elif(isinstance(imgs, PIL.Image.Image)):
        imgs = torch.stack([transform(imgs)])
    elif(type(imgs)==list):
        res = model(imgs)
        return [ res.pandas().xyxy[i].values[:,:6].tolist() for i in range(0,len(imgs))]
        imgList = []
        if(type(imgs[0])==str):
            for i in imgs:
                imgList.append(transform(Image.open(i)))
            imgs = torch.stack(imgList)
        elif(isinstance(imgs[0], PIL.Image.Image)):
            imgList = [transform(i) for i in imgs]
            imgs = torch.stack(imgList)
        elif(type(imgs[0])==np.ndarray):
            res = model(imgs)
            return [ res.pandas().xyxy[i].values[:,:6].tolist() for i in range(0,len(imgs))]
            model([cvimg,cvimg]).pandas().xyxy[0].values.tolist()
            
    else:
        try:
            res = model(imgs)
            resList = res.pandas().xyxy[0]
            # print(resList.values)
            return resList.values[:,:6].tolist()
        except:
            raise Exception("Not a Valid Image or Image Batch")
    res = model(imgs)
    res = non_max_suppression(res,conf,nms_threshold)
    results = []
    for r in res:
        if 0 not in r.shape:
            r = r.cpu()
            r = r.tolist()
            results.append(r)
        else:
            results.append([])
    return results


def readSignalBoard(model,frame):
    result = model.readtext(frame)
    speed = result[0][1]
    acc = result[0][2]
    return(speed, acc)

# Link can be of type both the video link or the image df/cv2.image/PIL Image
def processFeed(link,type,YoloModel=None,reader=None):
    global currentSpeedLimit
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    if(YoloModel == None):
        YoloModel = torch.hub.load('ultralytics/yolov5','custom',"./yolov5/runs/train/final/weights/last.pt").to(device)
    if(reader==None):
        reader = easyocr.Reader(['en'])
    if(type=="video"):
        cap = cv2.VideoCapture(link)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print("NUMBER OF FRAMES IN THE GIVEN VIDEO: ",length)
        print("FPS of given video:", cap.get(cv2.CAP_PROP_FPS))
    elif(type=="feed"):
        pass
    elif(type=="image"):
        frame = link
        detections = YoloDetection(YoloModel,frame)
        for d in detections:
            # print(d)
            [xmin,ymin,xmax,ymax,confidence, clsName] = d
            xmin = int(xmin)
            xmax = int(xmax)
            ymin = int(ymin)
            ymax = int(ymax)
            croppedFrame = frame[ymin:ymax, xmin:xmax]
            if(clsName == 8):
                speedLimit, acc = readSignalBoard(reader,croppedFrame)
                currentSpeedLimit = speedLimit
                # print("speed limit set..........")
        return currentSpeedLimit
    res, frame = cap.read()
    frameId = 0
    while(res):
        # st = time.time()
        detections = YoloDetection(YoloModel,frame)
        for d in detections:
            [xmin,ymin,xmax,ymax,confidence, clsName] = d
            xmin = int(xmin)
            xmax = int(xmax)
            ymin = int(ymin)
            ymax = int(ymax)
            croppedFrame = frame[ymin:ymax, xmin:xmax]
            if(clsName == 8):
                speedLimit, acc = readSignalBoard(reader,croppedFrame)
                currentSpeedLimit = speedLimit
                # print(speedLimit,acc)
                # print(frameId,confidence)
        frameId += 1
        res, frame = cap.read()

        if(frameId % 50 == 0):
            print("Current Frame Id: ",frameId,", currentSpeedLimit: ",currentSpeedLimit)


        # print("process Time",time.time()-st)
    del(YoloModel,reader)


def Speed_Cal(time):
    #Here i converted m to Km and second to hour then divison to reach Speed in this form (KM/H) 
    #the 9.144 is distance of free space between two lines # found in https://news.osu.edu/slow-down----those-lines-on-the-road-are-longer-than-you-think/
    #i know that the 9.144 is an standard and my video may not be that but its like guess and its need Field research
    try:
        Speed = (9.144*3600)/(time*1000)
        return Speed
    except ZeroDivisionError:
        print (5)


def ProcessVideoWithSpeedDetection(cascade_src,video_src):
    global currentSpeedLimit
    print("Displaying speed of train at present time instance")
    ax1=30
    ay=300
    ax2=900
    #line b
    bx1=30
    by=400
    bx2=900
    #car num
    i = 1
    start_time = time.time()
    #video ....
    cap = cv2.VideoCapture(video_src)
    car_cascade = cv2.CascadeClassifier(cascade_src)

    current_frame = 0
    start_frame = 0
    current_fps = cap.get(cv2.CAP_PROP_FPS)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    YoloModel = torch.hub.load('ultralytics/yolov5','custom',"./yolov5/runs/train/final/weights/last.pt").to(device)
    reader = easyocr.Reader(['en'])
    while True:
        current_frame += 1
        ret, img = cap.read()
        if (type(img) == type(None)):
            break
        # processFeed(img,"image",YoloModel,reader)
        # if(current_frame%50==0):
        #     print(current_frame, currentSpeedLimit)
        # print(type(img),current_frame)

        #bluring to have exacter detection
        SLD_Img = img.copy()
        blurred = cv2.blur(img,ksize=(15,15))
        gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
        cars = car_cascade.detectMultiScale(gray, 1.1, 2)
        
        #line a #i know road has got 
        cv2.line(img,(ax1,ay),(ax2,ay),(255,0,0),2)
        #line b
        cv2.line(img,(bx1,by),(bx2,by),(255,0,0),2)
            
        for (x,y,w,h) in cars:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)   
            cv2.circle(img,(int((x+x+w)/2),int((y+y+h)/2)),1,(0,255,0),-1)
            
            while int(ay) == int((y+y+h)/2):
                start_time = time.time()
                start_frame = current_frame
                break
                
            while int(ay) <= int((y+y+h)/2):
                # Checking if the cascaded pixels reached the current lower line
                if int(by) <= int((y+y+h)/2)&int(by+10) >= int((y+y+h)/2):
                    cv2.line(img,(bx1,by),(bx2,by),(0,255,0),2)
                    Speed = Speed_Cal((current_frame-start_frame)/current_fps)
                    now = datetime.datetime.now()
                    print("Time: " + str(now.time()) +"   Speed: "+str(Speed) + " KM/H", "Speed Limit: ", currentSpeedLimit)
                    try:
                        if(Speed > int(currentSpeedLimit)):
                            print("WARNING: current speed over current speed limit...")
                            print("Speed: ", Speed, " Speed Limit: ", int(currentSpeedLimit))
                    except:
                        pass
                    i = i + 1
                    cv2.putText(img, "Speed: "+str(Speed)+"KM/H", (x,y-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),3);
                    # cv2.putText(img, "Speed: "+str(Speed)+"KM/H"+" Limit: "+str(currentSpeedLimit), (x,y-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),3);
                    break
                else :
                    cv2.putText(img, "Calculating", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),3)
                    break
                    
        processFeed(SLD_Img,"image",YoloModel,reader)
        # cv2.imshow('video', img)
        
        # if cv2.waitKey(33) == 27:
        #     break

        ### GETTING THE SPEED LIMIT UPDATED ###
        
    # cv2.destroyAllWindows()
    pass




In [4]:
ProcessVideoWithSpeedDetection(cascade_src,video_src)

Displaying speed of train at present time instance


Using cache found in /home/tcestudent/.cache/torch/hub/ultralytics_yolov5_master
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show() image displays
OpenCV(4.5.4) /tmp/pip-req-build-jpmv6t9_/opencv/modules/highgui/src/window.cpp:1274: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'

YOLOv5 🚀 2022-11-26 Python-3.7.13 torch-1.13.0+cu117 CUDA:0 (Quadro RTX 8000, 48601MiB)

Fusing layers... 
Model summary: 157 layers, 7042489 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


Time: 08:01:41.864020   Speed: 141.07885714285715 KM/H Speed Limit:  500
Time: 08:02:03.032044   Speed: 7.900416 KM/H Speed Limit:  50
Time: 08:02:03.641354   Speed: 7.655441860465117 KM/H Speed Limit:  50
Time: 08:02:12.543835   Speed: 37.98276923076923 KM/H Speed Limit:  50
Time: 08:02:13.014044   Speed: 987.552 KM/H Speed Limit:  50
Speed:  987.552  Speed Limit:  50
Time: 08:02:13.944856   Speed: 141.07885714285715 KM/H Speed Limit:  50
Speed:  141.07885714285715  Speed Limit:  50
Time: 08:02:18.598287   Speed: 493.776 KM/H Speed Limit:  50
Speed:  493.776  Speed Limit:  50
Time: 08:02:18.897280   Speed: 246.888 KM/H Speed Limit:  50
Speed:  246.888  Speed Limit:  50
Time: 08:02:51.538888   Speed: 54.864000000000004 KM/H Speed Limit:  80
Time: 08:02:57.971593   Speed: 51.976421052631586 KM/H Speed Limit:  80
Time: 08:02:58.889543   Speed: 39.50208 KM/H Speed Limit:  80
Time: 08:02:59.686716   Speed: 32.9184 KM/H Speed Limit:  80
Time: 08:03:24.568868   Speed: 41.148 KM/H Speed Limit